## Load data

In [16]:
import pandas as pd
import ast
import numpy as np

# Load in data
admissions = 'tedsa_puf_2019.csv'
df = pd.read_csv(f'../../Downloads/{admissions}')

## Compare SUB1 groups with DSMCRIT

In [17]:
sub1 = df['SUB1']
for i in [1,2,3,4,8,9,10,11,12,13,14,15,16,17,18,19]:
    sub1 = sub1.replace(i, 'Other')
# sub1 = sub1.replace(-9, np.nan)

dsmcrit = df['DSMCRIT']
for i in [1,2,3,4,6,7,8,9,10,11,13,14,15,16,17,18,19]:
    dsmcrit = dsmcrit.replace(i, 'Other')
# dsmcrit = dsmcrit.replace(-9, np.nan)

In [18]:
df4 = pd.DataFrame()
df4['SUB1'] = sub1
df4['DSMCRIT'] = dsmcrit

df4.value_counts()

print('See "SUB1 DSMCRIT overlap.xlsx" file for a better breakdown')

See "SUB1 DSMCRIT overlap.xlsx" file for a better breakdown


## Filter out select rows and columns

In [19]:
df.columns

Index(['ADMYR', 'CASEID', 'STFIPS', 'CBSA2010', 'EDUC', 'MARSTAT', 'SERVICES',
       'DETCRIM', 'NOPRIOR', 'PSOURCE', 'ARRESTS', 'EMPLOY', 'METHUSE',
       'PSYPROB', 'PREG', 'GENDER', 'VET', 'LIVARAG', 'DAYWAIT', 'DSMCRIT',
       'AGE', 'RACE', 'ETHNIC', 'DETNLF', 'PRIMINC', 'SUB1', 'SUB2', 'SUB3',
       'ROUTE1', 'ROUTE2', 'ROUTE3', 'FREQ1', 'FREQ2', 'FREQ3', 'FRSTUSE1',
       'FRSTUSE2', 'FRSTUSE3', 'HLTHINS', 'PRIMPAY', 'FREQ_ATND_SELF_HELP',
       'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'METHFLG', 'OPSYNFLG',
       'PCPFLG', 'HALLFLG', 'MTHAMFLG', 'AMPHFLG', 'STIMFLG', 'BENZFLG',
       'TRNQFLG', 'BARBFLG', 'SEDHPFLG', 'INHFLG', 'OTCFLG', 'OTHERFLG',
       'DIVISION', 'REGION', 'IDU', 'ALCDRUG'],
      dtype='object')

In [20]:
# Get count of original number of rows
old_rows = len(df)

# Drop Puerto Rico
df = df[df['STFIPS'] != 72]

# Drop defined columns
columns_to_drop = ['ADMYR', 'CASEID', 'DETNLF', 'DETCRIM']  # consider including 'STFIPS', 'CBSA2010'
df = df.drop(columns=columns_to_drop)
print(f'Dropped {len(columns_to_drop)} columns ({len(df.columns)} remain)')

# Drop values where dependent variable is unknown
df = df[df['METHUSE'] != -9]

# Only keep patients admitted with self-described use of an opioid as their primary substance use (i.e., SUB1 = 5, 6, or 7)
df = df[df['SUB1'].between(5, 7)]
new_rows = len(df)
percent_change = round(100*(old_rows-new_rows)/old_rows, 1)
print(f'Dropped {old_rows-new_rows} observations or {percent_change}% of the data ({new_rows} rows remain)')

df = df.reset_index(drop='index')

Dropped 4 columns (58 remain)
Dropped 1341403 observations or 71.9% of the data (522964 rows remain)


In [21]:
df

,STFIPS,CBSA2010,EDUC,MARSTAT,SERVICES,NOPRIOR,PSOURCE,ARRESTS,EMPLOY,METHUSE,...,TRNQFLG,BARBFLG,SEDHPFLG,INHFLG,OTCFLG,OTHERFLG,DIVISION,REGION,IDU,ALCDRUG
0,2,-9,2,1,7,3,1,0,3,2,...,0,0,0,0,0,0,9,4,1,3
1,2,-9,3,1,7,2,3,0,4,1,...,0,0,0,0,0,0,9,4,1,2
2,2,-9,1,2,7,5,2,0,4,1,...,0,0,0,0,0,0,9,4,1,2
3,2,-9,3,1,7,3,3,0,4,1,...,0,0,0,0,0,0,9,4,1,2
4,2,-9,3,1,7,1,2,0,2,1,...,0,0,0,0,0,0,9,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522959,56,-9,-9,2,6,0,6,0,3,2,...,0,0,0,0,0,0,8,4,0,2
522960,56,-9,-9,2,6,2,7,0,3,2,...,0,0,0,0,0,0,8,4,0,2
522961,56,-9,3,1,7,3,2,0,3,2,...,0,0,1,0,0,0,8,4,0,3
522962,56,-9,1,1,7,0,7,0,4,2,...,0,0,0,0,0,0,8,4,0,2


## Make dataset human-readable

In [22]:
# Load in variable dictionary
with open('variable_dictionary.txt') as file:
    variable_dict_string = file.read()
    variable_dict = ast.literal_eval(variable_dict_string)

# Rename entries in column according to dictionary
df2 = df.copy()
for col, col_dict in variable_dict.items():
    for old_value, new_value in variable_dict[col].items():
        df2[col] = df2[col].replace(old_value, new_value)

# Rename "-9" values as "Unknown"
for col in df2.columns:
    df2[col] = df2[col].replace(-9, 'Unknown')

FileNotFoundError: [Errno 2] No such file or directory: 'variable_dictionary.txt'

In [ ]:
df2

## Make machine-readable dataset
todo clean up this section

In [ ]:
# Remove dependent variable
df2['METHUSE'] = df2['METHUSE'].replace('MethUse', 1)
df2['METHUSE'] = df2['METHUSE'].replace('NoMethUse', 0)

In [ ]:
df3 = df2.copy()

# Convert categorical variables to dummy variables
df3 = pd.get_dummies(df3)

# Add intercept
df3.insert(0, 'Intercept', 1)

# Save dataframe to csv and show below
df3.to_csv('data.csv', index=False)
df3

## Run linear regression to test out the data

In [ ]:
from sklearn.model_selection import train_test_split

# Define independent variables X and dependent variable y
y = df3['METHUSE']
X = df3.drop(columns='METHUSE')

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=24)

In [ ]:
# Load modules and data
import statsmodels
import statsmodels.api as sm

# Fit and summarize OLS model
model = sm.OLS(y_train, X_train)
results = model.fit()

y_hat = results.predict(X_test)
mse = statsmodels.tools.eval_measures.mse(y_test, y_hat)

print('MSE:', mse)
print(results.summary())